In [17]:
import json
databaseFile = "./database.json"
with open(databaseFile, encoding='utf-8') as f:
    database = [json.loads(l.strip()) for l in f.readlines()] 

In [21]:
directorNames = [dir['name'] for dir in database]

In [33]:
import random
links = ['https://en.wikipedia.org/wiki/' + name.replace(' ', '_') for name in random.sample(directorNames, 20)]

In [121]:
from urllib.request import (urlopen, urlparse, urlunparse, urlretrieve)
from bs4 import BeautifulSoup as bs
import re
from nltk.tokenize import sent_tokenize

In [122]:
def process_content(link):
    soup = bs(urlopen(link), "lxml")
    for script in soup(["script", "style", "table","ul"]):
        script.decompose()
    main = soup.findAll("p")
    identFilter = re.sub("[\n\r\t]{1,}", "\n", ''.join(soup.get_text())).strip()
    shortFilter = list(filter(lambda s: len(s) > 50 and not s.startswith('^ '), identFilter.split('\n')))
    text = ' '.join(list(map(lambda s: re.sub('\[[0-9]+\]', '',s), shortFilter)))
    return sent_tokenize(text)

In [130]:
def extract_facts(directorObj, facts):
    name = directorObj['name']
    films = [film['name'] for film in directorObj['films']]
    link = 'https://en.wikipedia.org/wiki/' + name.replace(' ', '_')
    sentences =  process_content(link)
    for sent in sentences:
        for f in films:
            if f in sent:
                facts.append({name:sent})
                break

In [131]:
facts = []
for directorObj in random.sample(database, 20):
    extract_facts(directorObj, facts)

In [135]:
jsonFacts = [json.dumps(fact) for fact in facts]

In [149]:
factsWithYear = list(filter(lambda s: re.search('[1-3][0-9]{3}', s), jsonFacts))

In [151]:
with open('./extractedFactsWithYear.json', 'w' , encoding='utf-8') as f:
    f.write('\n'.join(factsWithYear))

In [157]:
otherNames = set([list(fact.keys())[0] for fact in facts])

In [158]:
distinct = list(filter(lambda d: d['name'] not in otherNames, database))

In [159]:
facts2 = []
for directorObj in distinct:
    extract_facts(directorObj, facts2)

HTTPError: HTTP Error 404: Not Found

In [161]:
jsonFacts2 = [json.dumps(fact) for fact in facts2]
factsWithYear2 = list(filter(lambda s: re.search('[1-3][0-9]{3}', s), jsonFacts2))
with open('./extractedFactsWithYear_notEdited.json', 'w' , encoding='utf-8') as f:
    f.write('\n'.join(factsWithYear2))